In [ ]:
#@title Imports { vertical-output: true }
import pandas as pd
import json
import duckdb
import requests
import os
import pathlib

from ipytree import Tree, Node

try:
    from google.colab import data_table, output
    data_table.enable_dataframe_formatter()
    output.enable_custom_widget_manager()
except ImportError:
    # Not running in Colab, ignore
    pass

%pip install vl-convert-python


In [ ]:
#@title 2023 - RFL League Data Json [Rules,Scoring,Rosters]
league_data_json_2023 = {
  "league": "RFL",
  "season": 2023,
  "scoring_period": "Weekly",
  "roster_summary": {
    "total_roster": 30,
    "use_total": 17,
    "backups_total": 13
  },
  "roster_rules": {
    "QB": {
      "roster": 3,
      "use": 2,
      "backup": 1
    },
    "RB": {
      "roster": 4,
      "use": 2,
      "backup": 2
    },
    "WR": {
      "roster": 5,
      "use": 3,
      "backup": 2
    },
    "TE": {
      "roster": 2,
      "use": 1,
      "backup": 1
    },
    "K": {
      "roster": 2,
      "use": 1,
      "backup": 1
    },
    "DL": {
      "roster": 5,
      "use": 3,
      "backup": 2
    },
    "LB": {
      "roster": 4,
      "use": 2,
      "backup": 2
    },
    "DB": {
      "roster": 5,
      "use": 3,
      "backup": 2
    }
  },
  "scoring": {
    "QB": {
      "passing_yards": {
        "every": 100,
        "points": 1
      },
      "passings_tds": {
        "every": 1,
        "points": 1
      },
      "rushing_yards": {
        "every": 100,
        "points": 2
      },
      "rushing_tds": {
        "every": 1,
        "points": 3
      }
    },
    "RB": {
      "rushing_yards": {
        "every": 100,
        "points": 2
      },
      "rushing_tds": {
        "every": 1,
        "points": 3
      },
      "receiving_yards": {
        "every": 100,
        "points": 2
      },
      "receiving_tds": {
        "every": 1,
        "points": 3
      }
    },
    "WR": {
      "receiving_yards": {
        "every": 100,
        "points": 3
      },
      "receiving_tds": {
        "every": 1,
        "points": 3
      },
      "rushing_yards": {
        "every": 100,
        "points": 3
      },
      "rushing_tds": {
        "every": 1,
        "points": 3
      }
    },
    "TE": {
      "receiving_yards": {
        "every": 100,
        "points": 5
      },
      "receiving_tds": {
        "every": 1,
        "points": 3
      }
    },
    "K": {
      "fg_made": {
        "every": 1,
        "points": 2
      }
    },
    "DL": {
      "full_sack": {
        "every": 1,
        "points": 4
      },
      "half_sack": {
        "every": 0.5,
        "points": 2
      }
    },
    "LB": {
      "full_sack": {
        "every": 1,
        "points": 4
      },
      "half_sack": {
        "every": 0.5,
        "points": 2
      },
      "def_interceptions": {
        "every": 1,
        "points": 3
      },
      "tackles": {
        "every": 5,
        "points": 1
      }
    },
    "DB": {
      "def_interceptions": {
        "every": 1,
        "points": 7
      },
      "def_tds": {
        "every": 1,
        "points": 3
      }
    }
  },
  "teams": [
    {
      "team": "Bears",
      "owner": "Chris",
      "roster": {
        "QB": [
          "JOSH ALLEN",
          "KIRK COUSINS",
          "SAM HOWELL"
        ],
        "RB": [
          "CHRISTIAN MCCAFFREY",
          "DALVIN COOK",
          "James Conner",
          "Raheem Mostert"
        ],
        "WR": [
          "STEFON DIGGS",
          "AMARI COOPER",
          "CHRISTIAN KIRK",
          "SKYY MOORE",
          "VAN JEFFERSON"
        ],
        "TE": [
          "GEORGE KITTLE",
          "JELANI WOODS"
        ],
        "K": [
          "KA'IMI FAIRBAIRN",
          "MATT GAY"
        ],
        "DL": [
          "AARON DONALD",
          "MAXX CROSBY",
          "CHRIS JONES",
          "YANNICK NGAKOUE",
          "GEORGE KARLAFTIS"
        ],
        "LB": [
          "VON MILLER",
          "SHAQUIL BARRETT",
          "PRESTON SMITH",
          "BOYE MAFE"
        ],
        "DB": [
          "TREVON DIGGS",
          "DARON BLAND",
          "TALANOA HUFANGA",
          "DAXTON HILL",
          "ALONTAE TAYLOR"
        ]
      }
    },
    {
      "team": "Raiders",
      "owner": "Couto",
      "roster": {
        "QB": [
          "DEREK CARR",
          "DAK PRESCOTT",
          "JARED GOFF"
        ],
        "RB": [
          "Jonathan Taylor",
          "TONY POLLARD",
          "JAMES COOK",
          "Rashaad Penny"
        ],
        "WR": [
          "A.J. BROWN",
          "COOPER KUPP",
          "BRANDON AIYUK",
          "TREYLON BURKS",
          "ELIJAH MOORE"
        ],
        "TE": [
          "DALLAS GOEDERT",
          "DAVID NJOKU"
        ],
        "K": [
          "JUSTIN TUCKER",
          "DUSTIN HOPKINS"
        ],
        "DL": [
          "MYLES GARRETT",
          "CHANDLER JONES",
          "DEFOREST BUCKNER",
          "DRE'MONT JONES",
          "JOHN FRANKLIN-MYERS"
        ],
        "LB": [
          "T.J. WATT",
          "DANIELLE HUNTER",
          "JAELAN PHILLIPS",
          "UCHENNA NWOSU"
        ],
        "DB": [
          "MINKAH FITZPATRICK",
          "JUSTIN SIMMONS",
          "PAT SURTAIN II",
          "RIQ WOOLEN",
          "JEFF OKUDAH"
        ]
      }
    },
    {
      "team": "Eagles",
      "owner": "Marco",
      "roster": {
        "QB": [
          "AARON RODGERS",
          "TUA TAGOVAILOA",
          "BAKER MAYFIELD"
        ],
        "RB": [
          "AUSTIN EKELER",
          "NAJEE HARRIS",
          "Isiah Pacheco",
          "JAMAAL WILLIAMS"
        ],
        "WR": [
          "JUSTIN JEFFERSON",
          "MIKE EVANS",
          "CALVIN RIDLEY",
          "GABE DAVIS",
          "RASHOD BATEMAN"
        ],
        "TE": [
          "MARK ANDREWS",
          "COLE KMET"
        ],
        "K": [
          "TYLER BASS",
          "EVAN MCPHERSON"
        ],
        "DL": [
          "Jonathan Allen",
          "KWITY PAYE",
          "DENICO AUTRY",
          "DEXTER LAWRENCE",
          "BRANDON GRAHAM"
        ],
        "LB": [
          "ZA'DARIUS SMITH",
          "JOSH ALLEN",
          "JOSHUA UCHE",
          "DARRELL TAYLOR"
        ],
        "DB": [
          "Xavien Howard",
          "MARSHON LATTIMORE",
          "SAUCE GARDNER",
          "KYLE DUGGER",
          "DARRICK FORREST"
        ]
      }
    },
    {
      "team": "Cowboys",
      "owner": "Tony",
      "roster": {
        "QB": [
          "RUSSELL WILSON",
          "JORDAN LOVE",
          "RYAN TANNEHILL"
        ],
        "RB": [
          "NICK CHUBB",
          "Javonte Williams",
          "KENNETH WALKER III",
          "Ezekiel Elliott"
        ],
        "WR": [
          "DAVANTE ADAMS",
          "DK METCALF",
          "CEEDEE LAMB",
          "JAHAN DOTSON",
          "JOSH PALMER"
        ],
        "TE": [
          "PAT FREIERMUTH",
          "JAKE FERGUSON"
        ],
        "K": [
          "JASON MYERS",
          "GRAHAM GANO"
        ],
        "DL": [
          "BRIAN BURNS",
          "SAM HUBBARD",
          "JADEVEON CLOWNEY",
          "JERMAINE JOHNSON II",
          "SAM WILLIAMS"
        ],
        "LB": [
          "MATTHEW JUDON",
          "JUSTIN HOUSTON",
          "Rashan Gary",
          "ARNOLD EBIKETIE"
        ],
        "DB": [
          "KEVIN BYARD",
          "JORDAN WHITEHEAD",
          "JULIAN BLACKMON",
          "JOHN JOHNSON",
          "TRENT MCDUFFIE"
        ]
      }
    },
    {
      "team": "Colts",
      "owner": "Rizzo",
      "roster": {
        "QB": [
          "JOE BURROW",
          "DESHAUN WATSON",
          "BROCK PURDY"
        ],
        "RB": [
          "MILES SANDERS",
          "AARON JONES",
          "DAVID MONTGOMERY",
          "D'ANDRE SWIFT"
        ],
        "WR": [
          "JA'MARR CHASE",
          "DeVonta Smith",
          "AMON-RA ST. BROWN",
          "DJ MOORE",
          "MIKE WILLIAMS"
        ],
        "TE": [
          "GREG DULCICH",
          "Chig Okonkwo"
        ],
        "K": [
          "BRANDON MCMANUS",
          "JOEY SLYE"
        ],
        "DL": [
          "QUINNEN WILLIAMS",
          "GREG ROUSSEAU",
          "Jeffery Simmons",
          "CHRISTIAN BARMORE",
          "DARON PAYNE"
        ],
        "LB": [
          "LEONARD FLOYD",
          "KAYVON THIBODEAUX",
          "SHAQUILLE LEONARD",
          "Jeremiah Owusu-Koramoah"
        ],
        "DB": [
          "JAIRE ALEXANDER",
          "DERWIN JAMES",
          "C.J. Gardner-Johnson",
          "L'JARIUS SNEED",
          "AMANI HOOKER"
        ]
      }
    },
    {
      "team": "Steelers",
      "owner": "Super Dave",
      "roster": {
        "QB": [
          "PATRICK MAHOMES",
          "KENNY PICKETT",
          "TREY LANCE"
        ],
        "RB": [
          "CAM AKERS",
          "RACHAAD WHITE",
          "Brian Robinson",
          "KAREEM HUNT"
        ],
        "WR": [
          "TYREEK HILL",
          "Keenan Allen",
          "JAYLEN WADDLE",
          "DIONTAE JOHNSON",
          "KADARIUS TONEY"
        ],
        "TE": [
          "TRAVIS KELCE",
          "ZACH ERTZ"
        ],
        "K": [
          "GREG JOSEPH",
          "Wil Lutz"
        ],
        "DL": [
          "NICK BOSA",
          "A.J. EPENESA",
          "ARIK ARMSTEAD",
          "CAMERON HEYWARD",
          "JORDAN DAVIS"
        ],
        "LB": [
          "JOEY BOSA",
          "KHALIL MACK",
          "ALEX HIGHSMITH",
          "BOBBY WAGNER"
        ],
        "DB": [
          "Jessie Bates",
          "KENDALL FULLER",
          "A.J. TERRELL",
          "Derek Stingley Jr.",
          "KAIIR ELAM"
        ]
      }
    },
    {
      "team": "Vikings",
      "owner": "Danny",
      "roster": {
        "QB": [
          "LAMAR JACKSON",
          "JALEN HURTS",
          "JIMMY GAROPPOLO"
        ],
        "RB": [
          "SAQUON BARKLEY",
          "TRAVIS ETIENNE",
          "RHAMONDRE STEVENSON",
          "DAMIEN HARRIS"
        ],
        "WR": [
          "MICHAEL THOMAS",
          "TERRY MCLAURIN",
          "JuJu Smith-Schuster",
          "Marquise Brown",
          "TYLER LOCKETT"
        ],
        "TE": [
          "KYLE PITTS",
          "DALTON SCHULTZ"
        ],
        "K": [
          "MATT PRATER",
          "RILEY PATTERSON"
        ],
        "DL": [
          "TREY HENDRICKSON",
          "CHASE YOUNG",
          "CARL GRANDERSON",
          "ZACH ALLEN",
          "JERRY HUGHES"
        ],
        "LB": [
          "MICAH PARSONS",
          "HAASON REDDICK",
          "DAVID OJABO",
          "JAMES HOUSTON"
        ],
        "DB": [
          "TYRANN MATHIEU",
          "JAYCEE HORN",
          "JEVON HOLLAND",
          "DENZEL WARD",
          "KENNY MOORE II"
        ]
      }
    },
    {
      "team": "Jets",
      "owner": "Mario",
      "roster": {
        "QB": [
          "JUSTIN FIELDS",
          "MAC JONES",
          "DESMOND RIDDER"
        ],
        "RB": [
          "JOSH JACOBS",
          "BREECE HALL",
          "ALEXANDER MATTISON",
          "KHALIL HERBERT"
        ],
        "WR": [
          "MICHAEL PITTMAN",
          "NICO COLLINS",
          "DRAKE LONDON",
          "CHRIS OLAVE",
          "GEORGE PICKENS"
        ],
        "TE": [
          "EVAN ENGRAM",
          "MIKE GESICKI"
        ],
        "K": [
          "JASON SANDERS",
          "CHRIS BOSWELL"
        ],
        "DL": [
          "JOSH SWEAT",
          "SAMSON EBUKAM",
          "EMMANUEL OGBAH",
          "Dayo Odeyingbo",
          "Nnamdi Madubuike"
        ],
        "LB": [
          "Roquan Smith",
          "ODAFE OWEH",
          "TRAVON WALKER",
          "NICK BOLTON"
        ],
        "DB": [
          "DARIUS SLAY",
          "RASUL DOUGLAS",
          "KYLE HAMILTON",
          "ROCK YA-SIN",
          "LEVI WALLACE"
        ]
      }
    },
    {
      "team": "Bengals",
      "owner": "Mass",
      "roster": {
        "QB": [
          "JUSTIN HERBERT",
          "GENO SMITH",
          "DANIEL JONES"
        ],
        "RB": [
          "DERRICK HENRY",
          "J.K. DOBBINS",
          "Dameon Pierce",
          "SAMAJE PERINE"
        ],
        "WR": [
          "DEANDRE HOPKINS",
          "CHRIS GODWIN",
          "JERRY JEUDY",
          "DEEBO SAMUEL SR.",
          "GARRETT WILSON"
        ],
        "TE": [
          "DARREN WALLER",
          "TYLER HIGBEE"
        ],
        "K": [
          "DANIEL CARLSON",
          "EDDY PINEIRO"
        ],
        "DL": [
          "MONTEZ SWEAT",
          "FRANK CLARK",
          "CAMERON JORDAN",
          "MARCUS DAVENPORT",
          "DRAKE JACKSON"
        ],
        "LB": [
          "BRADLEY CHUBB",
          "DEVIN WHITE",
          "Baron Browning",
          "FOYE OLUOKUN"
        ],
        "DB": [
          "QUANDRE DIGGS",
          "Asante Samuel Jr.",
          "JORDAN POYER",
          "ANDRE CISCO",
          "TASHAUN GIPSON"
        ]
      }
    },
    {
      "team": "Packers",
      "owner": "Jumps",
      "roster": {
        "QB": [
          "MATTHEW STAFFORD",
          "TREVOR LAWRENCE",
          "KYLER MURRAY"
        ],
        "RB": [
          "ALVIN KAMARA",
          "JOE MIXON",
          "ANTONIO GIBSON",
          "AJ DILLON"
        ],
        "WR": [
          "TEE HIGGINS",
          "COURTLAND SUTTON",
          "BRANDIN COOKS",
          "CHRISTIAN WATSON",
          "Jameson Williams"
        ],
        "TE": [
          "T.J. HOCKENSON",
          "JUWAN JOHNSON"
        ],
        "K": [
          "YOUNGHOE KOO",
          "HARRISON BUTKER"
        ],
        "DL": [
          "DEMARCUS LAWRENCE",
          "JAVON HARGRAVE",
          "AIDAN HUTCHINSON",
          "Jonathan Greenard",
          "LEONARD WILLIAMS"
        ],
        "LB": [
          "Harold Landry III",
          "AZEEZ OJULARI",
          "Joe Tryon-Shoyinka",
          "FRANKIE LUVU"
        ],
        "DB": [
          "MARCUS WILLIAMS",
          "J.C. JACKSON",
          "KERBY JOSEPH",
          "Jalen Pitre",
          "GRANT DELPIT"
        ]
      }
    }
  ]
}

In [ ]:
#@title 2024 RFL League Data Json [Rules,Scoring,Rosters]
league_data_json_2024 = {
  "league": "RFL",
  "season": 2024,
  "scoring_period": "Weekly",
  "roster_summary": {
    "total_roster": 36,
    "use_total": 17,
    "backups_total": 19
  },
  "roster_rules": {
    "QB": {"roster": 3, "use": 2, "backup": 1},
    "RB": {"roster": 5, "use": 2, "backup": 3},
    "WR": {"roster": 6, "use": 3, "backup": 3},
    "TE": {"roster": 3, "use": 1, "backup": 2},
    "K":  {"roster": 2, "use": 1, "backup": 1},
    "DL": {"roster": 6, "use": 3, "backup": 3},
    "LB": {"roster": 5, "use": 2, "backup": 3},
    "DB": {"roster": 6, "use": 3, "backup": 3}
  },
  "scoring": {
    "QB": {
      "passing_yards": {
        "every": 100,
        "points": 1
      },
      "passings_tds": {
        "every": 1,
        "points": 1
      },
      "rushing_yards": {
        "every": 100,
        "points": 2
      },
      "rushing_tds": {
        "every": 1,
        "points": 3
      }
    },
    "RB": {
      "rushing_yards": {
        "every": 100,
        "points": 2
      },
      "rushing_tds": {
        "every": 1,
        "points": 3
      },
      "receiving_yards": {
        "every": 100,
        "points": 2
      },
      "receiving_tds": {
        "every": 1,
        "points": 3
      }
    },
    "WR": {
      "receiving_yards": {
        "every": 100,
        "points": 3
      },
      "receiving_tds": {
        "every": 1,
        "points": 3
      },
      "rushing_yards": {
        "every": 100,
        "points": 3
      },
      "rushing_tds": {
        "every": 1,
        "points": 3
      }
    },
    "TE": {
      "receiving_yards": {
        "every": 100,
        "points": 5
      },
      "receiving_tds": {
        "every": 1,
        "points": 3
      }
    },
    "K": {
      "fg_made": {
        "every": 1,
        "points": 2
      }
    },
    "DL": {
      "full_sack": {
        "every": 1,
        "points": 4
      },
      "half_sack": {
        "every": 0.5,
        "points": 2
      }
    },
    "LB": {
      "full_sack": {
        "every": 1,
        "points": 4
      },
      "half_sack": {
        "every": 0.5,
        "points": 2
      },
      "def_interceptions": {
        "every": 1,
        "points": 3
      },
      "tackles": {
        "every": 5,
        "points": 1
      }
    },
    "DB": {
      "def_interceptions": {
        "every": 1,
        "points": 7
      },
      "def_tds": {
        "every": 1,
        "points": 3
      }
    }
  },
  "teams": [
    {
      "team": "Vikings",
      "owner": "Danny",
      "roster": {
        "QB": ["LAMAR JACKSON","JALEN HURTS","SAM DARNOLD"],
        "RB": ["SAQUON BARKLEY","CHUBA HUBBARD","RHAMONDRE STEVENSON","TRAVIS ETIENNE","ZACK MOSS"],
        "WR": ["TERRY MCLAURIN","QUENTIN JOHNSTON","ZAY FLOWERS","MICHAEL WILSON","CURTIS SAMUEL","MARQUISE BROWN"],
        "TE": ["SAM LAPORTA","KYLE PITTS","DALTON SCHULTZ"],
        "K":  ["CAMERON DICKER","MATT PRATER"],
        "DL": ["TREY HENDRICKSON","CALIJAH KANCEY","CARL GRANDERSON","CHASE YOUNG","CHRISTIAN WILKINS","ALEX WRIGHT"],
        "LB": ["MICAH PARSONS","DERICK HALL","DARRELL TAYLOR","DAVID OJABO","JAMES HOUSTON"],
        "DB": ["BRIAN BRANCH","TYRANN MATHIEU","KENNY MOORE II","DENZEL WARD","JAYCEE HORN","JEVON HOLLAND"]
      }
    },
    {
      "team": "Colts",
      "owner": "Rizzo",
      "roster": {
        "QB": ["JOE BURROW","BROCK PURDY","DESHAUN WATSON"],
        "RB": ["JAHMYR GIBBS","DAVID MONTGOMERY","AARON JONES","D'ANDRE SWIFT","MILES SANDERS"],
        "WR": ["JA'MARR CHASE","AMON-RA ST. BROWN","DEVONTA SMITH","DJ MOORE","DONTAYVION WICKS","MIKE WILLIAMS"],
        "TE": ["DALTON KINCAID","CHIG OKONKWO","LUKE MUSGRAVE"],
        "K":  ["BRANDON AUBREY","JAKE MOODY"],
        "DL": ["LEONARD FLOYD","GREG ROUSSEAU","QUINNEN WILLIAMS","JEFFERY SIMMONS","DARON PAYNE","CHRISTIAN BARMORE"],
        "LB": ["NOLAN SMITH","JACK CAMPBELL","KAYVON THIBODEAUX","JEREMIAH OWUSU-KORAMOAH","SHAQUILLE LEONARD"],
        "DB": ["C.J. GARDNER-JOHNSON","AMANI HOOKER","JAIRE ALEXANDER","CHRISTIAN GONZALEZ","DERWIN JAMES","L'JARIUS SNEED"]
      }
    },
    {
      "team": "Packers",
      "owner": "Jumps",
      "roster": {
        "QB": ["KYLER MURRAY","MATTHEW STAFFORD","TREVOR LAWRENCE"],
        "RB": ["JOE MIXON","ALVIN KAMARA","RICO DOWDLE","KAREEM HUNT","JEROME FORD"],
        "WR": ["TEE HIGGINS","COURTLAND SUTTON","JAMESON WILLIAMS","PUKA NACUA","TANK DELL","CHRISTIAN WATSON"],
        "TE": ["JUWAN JOHNSON","NOAH FANT","T.J. HOCKENSON"],
        "K":  ["YOUNGHOE KOO","HARRISON BUTKER"],
        "DL": ["Jonathan GREENARD","LEONARD WILLIAMS","WILL MCDONALD IV","AIDAN HUTCHINSON","BRYCE HUFF","JAVON HARGRAVE"],
        "LB": ["FRANKIE LUVU","HAROLD LANDRY III","BYRON YOUNG","AZEEZ OJULARI","JOE TRYON-SHOYINKA"],
        "DB": ["KERBY JOSEPH","JAYLON JOHNSON","JALEN PITRE","JOEY PORTER JR.","DARIOUS WILLIAMS","MARCUS WILLIAMS"]
      }
    },
    {
      "team": "Eagles",
      "owner": "Marco",
      "roster": {
        "QB": ["BAKER MAYFIELD","AARON RODGERS","TUA TAGOVAILOA"],
        "RB": ["ZACH CHARBONNET","NAJEE HARRIS","AUSTIN EKELER","JALEEL MCLAUGHLIN","ISIAH PACHECO"],
        "WR": ["JUSTIN JEFFERSON","MIKE EVANS","CALVIN RIDLEY","JOSH DOWNS","RASHEE RICE","GABE DAVIS"],
        "TE": ["MARK ANDREWS","TUCKER KRAFT","COLE KMET"],
        "K":  ["TYLER BASS","EVAN MCPHERSON"],
        "DL": ["WILL ANDERSON JR.","DEXTER LAWRENCE","ZA'DARIUS SMITH","JALEN CARTER","JONATHAN ALLEN","DENICO AUTRY"],
        "LB": ["JONATHON COOPER","Josh Hines-Allen","RASHAN GARY","LUKAS VAN NESS","Joshua UCHE"],
        "DB": ["CAM TAYLOR-BRITT","SAUCE GARDNER","JI'AYIR BROWN","MARSHON LATTIMORE","KYLE DUGGER","DEONTE BANKS"]
      }
    },
    {
      "team": "Raiders",
      "owner": "Couto",
      "roster": {
        "QB": ["JARED GOFF","DEREK CARR","DAK PRESCOTT"],
        "RB": ["JAMES COOK","KYREN WILLIAMS","JONATHAN TAYLOR","CHASE BROWN","TONY POLLARD"],
        "WR": ["A.J. BROWN","JAYDEN REED","COOPER KUPP","KHALIL SHAKIR","DEMARIO DOUGLAS","BRANDON AIYUK"],
        "TE": ["DAVID NJOKU","DALLAS GOEDERT","DANIEL BELLINGER"],
        "K":  ["JUSTIN TUCKER","DUSTIN HOPKINS"],
        "DL": ["MYLES GARRETT","KOBIE TURNER","BRYAN BRESEE","DEFOREST BUCKNER","DEVONTE WYATT","KEION WHITE"],
        "LB": ["T.J. WATT","DANIELLE HUNTER","FRED WARNER","YAYA DIABY","JAELAN PHILLIPS"],
        "DB": ["PAT SURTAIN II","RIQ WOOLEN","JUSTIN SIMMONS","MINKAH FITZPATRICK","DEVON WITHERSPOON","TYSON CAMPBELL"]
      }
    },
    {
      "team": "Jets",
      "owner": "Mario",
      "roster": {
        "QB": ["C.J. STROUD","JUSTIN FIELDS","WILL LEVIS"],
        "RB": ["JOSH JACOBS","BREECE HALL","ALEXANDER MATTISON","DEVIN SINGLETARY","ZAMIR WHITE"],
        "WR": ["DRAKE LONDON","NICO COLLINS","GEORGE PICKENS","MICHAEL PITTMAN","ROMEO DOUBS","CHRIS OLAVE"],
        "TE": ["MIKE GESICKI","TYLER CONKLIN","EVAN ENGRAM"],
        "K":  ["CHRIS BOSWELL","JASON SANDERS"],
        "DL": ["ZACH SIELER","JOSH SWEAT","Nnamdi MADUBUIKE","DEMARCUS LAWRENCE","TYQUAN LEWIS","LARRY OGUNJOBI"],
        "LB": ["TRAVON WALKER","ODAFE OWEH","ROQUAN SMITH","PATRICK QUEEN","AZEEZ AL-SHAAIR"],
        "DB": ["XAVIER MCKINNEY","JULIAN LOVE","KYLE HAMILTON","CHARVARIUS WARD","JALEN THOMPSON","RASUL DOUGLAS"]
      }
    },
    {
      "team": "Steelers",
      "owner": "Dave",
      "roster": {
        "QB": ["PATRICK MAHOMES","BRYCE YOUNG","TREY LANCE"],
        "RB": ["RACHAAD WHITE","BRIAN ROBINSON","CAM AKERS","Jaylen WARREN","GUS EDWARDS"],
        "WR": ["JORDAN ADDISON","TYREEK HILL","KEENAN ALLEN","JAKOBI MEYERS","JAYLEN WADDLE","DIONTAE JOHNSON"],
        "TE": ["JONNU SMITH","ZACH ERTZ","TRAVIS KELCE"],
        "K":  ["WIL LUTZ","JAKE ELLIOTT"],
        "DL": ["NICK BOSA","CAMERON HEYWARD","A.J. EPENESA","MICHAEL DANNA","ED OLIVER","ARIK ARMSTEAD"],
        "LB": ["KYLE VAN NOY","BOBBY WAGNER","ALEX HIGHSMITH","KHALIL MACK","JOEY BOSA"],
        "DB": ["DEREK STINGLEY JR.","JESSIE BATES","PAULSON ADEBO","A.J. TERRELL","KAIIR ELAM","KENDALL FULLER"]
      }
    },
    {
      "team": "Bengals",
      "owner": "Mass",
      "roster": {
        "QB": ["GENO SMITH","JUSTIN HERBERT","DANIEL JONES"],
        "RB": ["DERRICK HENRY","BIJAN ROBINSON","J.K. DOBBINS","KENDRE MILLER","TY CHANDLER"],
        "WR": ["GARRETT WILSON","JAXON SMITH-NJIGBA","JERRY JEUDY","DEEBO SAMUEL SR.","DEANDRE HOPKINS","CHRIS GODWIN JR."],
        "TE": ["TREY MCBRIDE","ISAIAH LIKELY","HAYDEN HURST"],
        "K":  ["DANIEL CARLSON","BLAKE GRUPE"],
        "DL": ["MONTEZ SWEAT","CAMERON JORDAN","MARCUS DAVENPORT","TYREE WILSON","ZACH HARRISON","ISAIAH MCGUIRE"],
        "LB": ["BRADLEY CHUBB","FOYE OLUOKUN","BARON BROWNING","TERREL BERNARD","B.J. OJULARI"],
        "DB": ["JORDAN POYER","ASANTE SAMUEL JR.","QUANDRE DIGGS","TYRIQUE STEVENSON","MARTIN EMERSON","ANDRE CISCO"]
      }
    },
    {
      "team": "Bears",
      "owner": "Chris",
      "roster": {
        "QB": ["JOSH ALLEN","KIRK COUSINS","SAM HOWELL"],
        "RB": ["CHRISTIAN MCCAFFREY","RAHEEM MOSTERT","JAMES CONNER","DALVIN COOK","DE'VON ACHANE"],
        "WR": ["STEFON DIGGS","AMARI COOPER","CHRISTIAN KIRK","VAN JEFFERSON","RASHID SHAHEED","GREG DORTCH"],
        "TE": ["GEORGE KITTLE","JELANI WOODS","COLBY PARKINSON"],
        "K":  ["KA'IMI FAIRBAIRN","MATT GAY"],
        "DL": ["MAXX CROSBY","CHRIS JONES","GEORGE KARLAFTIS","FELIX ANUDIKE-UZOMAH","YANNICK NGAKOUE","ALIM MCNEILL"],
        "LB": ["HAASON REDDICK","SHAQUIL BARRETT","PRESTON SMITH","BOYE MAFE","TULI TUIPULOTU"],
        "DB": ["TREVON DIGGS","DARON BLAND","TALANOA HUFANGA","DAXTON HILL","ALONTAE TAYLOR","GENO STONE"]
      }
    },
    {
      "team": "Cowboys",
      "owner": "Tony",
      "roster": {
        "QB": ["RUSSELL WILSON","JORDAN LOVE","ANTHONY RICHARDSON"],
        "RB": ["NICK CHUBB","KENNETH WALKER III","JAVONTE WILLIAMS","TYJAE SPEARS","TANK BIGSBY"],
        "WR": ["DAVANTE ADAMS","DK METCALF","CEEDEE LAMB","JAHAN DOTSON","JOSH PALMER","MARVIN MIMS JR."],
        "TE": ["PAT FREIERMUTH","JAKE FERGUSON","CADE OTTON"],
        "K":  ["JASON MYERS","GRAHAM GANO"],
        "DL": ["KWITY PAYE","SAM HUBBARD","MALCOLM KOONCE","DORANCE ARMSTRONG","MYLES MURPHY","ISAIAH FOSKEY"],
        "LB": ["MATTHEW JUDON","BRIAN BURNS","JADEVEON CLOWNEY","JERMAINE JOHNSON II","ARNOLD EBIKETIE"],
        "DB": ["KEVIN BYARD","JORDAN WHITEHEAD","JULIAN BLACKMON","JOHN JOHNSON","TRENT MCDUFFIE","QUAN MARTIN"]
      }
    }
  ]
}

In [ ]:
#@title 2025 RFL League Data Json [Rules,Scoring,Rosters]
league_data_json = {
  "league": "RFL",
  "season": 2025,
  "scoring_period": "Weekly",
  "roster_summary": {
    "total_roster": 36,
    "use_total": 17,
    "backups_total": 19
  },
  "roster_rules": {
    "QB": {
      "roster": 3,
      "use": 2,
      "backup": 1
    },
    "RB": {
      "roster": 5,
      "use": 2,
      "backup": 3
    },
    "WR": {
      "roster": 6,
      "use": 3,
      "backup": 3
    },
    "TE": {
      "roster": 3,
      "use": 1,
      "backup": 2
    },
    "K": {
      "roster": 2,
      "use": 1,
      "backup": 1
    },
    "DL": {
      "roster": 6,
      "use": 3,
      "backup": 3
    },
    "LB": {
      "roster": 5,
      "use": 2,
      "backup": 3
    },
    "DB": {
      "roster": 6,
      "use": 3,
      "backup": 3
    }
  },
  "scoring": {
    "QB": {
      "passing_yards": {
        "every": 100,
        "points": 1
      },
      "passings_tds": {
        "every": 1,
        "points": 1
      },
      "rushing_yards": {
        "every": 100,
        "points": 2
      },
      "rushing_tds": {
        "every": 1,
        "points": 3
      }
    },
    "RB": {
      "rushing_yards": {
        "every": 100,
        "points": 2
      },
      "rushing_tds": {
        "every": 1,
        "points": 3
      },
      "receiving_yards": {
        "every": 100,
        "points": 2
      },
      "receiving_tds": {
        "every": 1,
        "points": 3
      }
    },
    "WR": {
      "receiving_yards": {
        "every": 100,
        "points": 3
      },
      "receiving_tds": {
        "every": 1,
        "points": 3
      },
      "rushing_yards": {
        "every": 100,
        "points": 3
      },
      "rushing_tds": {
        "every": 1,
        "points": 3
      }
    },
    "TE": {
      "receiving_yards": {
        "every": 100,
        "points": 5
      },
      "receiving_tds": {
        "every": 1,
        "points": 3
      }
    },
    "K": {
      "fg_made": {
        "every": 1,
        "points": 2
      }
    },
    "DL": {
      "full_sack": {
        "every": 1,
        "points": 4
      },
      "half_sack": {
        "every": 0.5,
        "points": 2
      }
    },
    "LB": {
      "full_sack": {
        "every": 1,
        "points": 4
      },
      "half_sack": {
        "every": 0.5,
        "points": 2
      },
      "def_interceptions": {
        "every": 1,
        "points": 3
      },
      "tackles": {
        "every": 5,
        "points": 1
      }
    },
    "DB": {
      "def_interceptions": {
        "every": 1,
        "points": 7
      },
      "def_tds": {
        "every": 1,
        "points": 3
      }
    }
  },
  "teams": [
    {
      "team": "Bears",
      "owner": "Chris",
      "roster": {
        "QB": [
          "JOSH ALLEN",
          "GENO SMITH",
          "SPENCER RATTLER"
        ],
        "RB": [
          "CHRISTIAN MCCAFFREY",
          "DE'VON ACHANE",
          "JAMES CONNER",
          "RAHEEM MOSTERT",
          "ROSCHON JOHNSON"
        ],
        "WR": [
          "STEFON DIGGS",
          "BRIAN THOMAS JR.",
          "CHRISTIAN KIRK",
          "RASHID SHAHEED",
          "RASHOD BATEMAN",
          "ADONAI MITCHELL"
        ],
        "TE": [
          "GEORGE KITTLE",
          "JELANI WOODS",
          "THEO JOHNSON"
        ],
        "K": [
          "KA'IMI FAIRBAIRN",
          "CHAD RYLAND"
        ],
        "DL": [
          "MAXX CROSBY",
          "CHRIS JONES",
          "GEORGE KARLAFTIS",
          "FELIX ANUDIKE-UZOMAH",
          "ALIM MCNEILL",
          "MORO OJOMO"
        ],
        "LB": [
          "HAASON REDDICK",
          "SHAQUIL BARRETT",
          "BOYE MAFE",
          "TULI TUIPULOTU",
          "YANNICK NGAKOUE"
        ],
        "DB": [
          "TREVON DIGGS",
          "DARON BLAND",
          "TALANOA HUFANGA",
          "ALONTAE TAYLOR",
          "MARLON HUMPHREY",
          "GENO STONE"
        ]
      }
    },
    {
      "team": "Raiders",
      "owner": "Couto",
      "roster": {
        "QB": [
          "DAK PRESCOTT",
          "JARED GOFF",
          "MICHAEL PENIX JR."
        ],
        "RB": [
          "JONATHAN TAYLOR",
          "JAMES COOK",
          "TONY POLLARD",
          "CHASE BROWN",
          "BRAELON ALLEN"
        ],
        "WR": [
          "A.J. BROWN",
          "COOPER KUPP",
          "JAYDEN REED",
          "KHALIL SHAKIR",
          "RICKY PEARSALL",
          "JAUAN JENNINGS"
        ],
        "TE": [
          "BROCK BOWERS",
          "DALLAS GOEDERT",
          "DAVID NJOKU"
        ],
        "K": [
          "JOSHUA KARTY",
          "WILL REICHARD"
        ],
        "DL": [
          "MYLES GARRETT",
          "DANIELLE HUNTER",
          "KOBIE TURNER",
          "KEION WHITE",
          "BRYAN BRESEE",
          "DEVONTE WYATT"
        ],
        "LB": [
          "T.J. WATT",
          "JAELAN PHILLIPS",
          "YAYA DIABY",
          "CHOP ROBINSON",
          "KADEN ELLISS"
        ],
        "DB": [
          "Pat Surtain II",
          "Riq Woolen",
          "DEVON WITHERSPOON",
          "TYSON CAMPBELL",
          "KAMARI LASSITER",
          "SAUCE GARDNER"
        ]
      }
    },
    {
      "team": "Eagles",
      "owner": "Marco",
      "roster": {
        "QB": [
          "BAKER MAYFIELD",
          "TUA TAGOVAILOA",
          "BO NIX"
        ],
        "RB": [
          "NAJEE HARRIS",
          "ISIAH PACHECO",
          "KYREN WILLIAMS",
          "ZACH CHARBONNET",
          "NICK CHUBB"
        ],
        "WR": [
          "JUSTIN JEFFERSON",
          "MIKE EVANS",
          "CALVIN RIDLEY",
          "RASHEE RICE",
          "JOSH DOWNS",
          "MICHAEL PITTMAN"
        ],
        "TE": [
          "MARK ANDREWS",
          "TUCKER KRAFT",
          "DALTON SCHULTZ"
        ],
        "K": [
          "JAKE BATES",
          "CAM LITTLE"
        ],
        "DL": [
          "WILL ANDERSON JR.",
          "RASHAN GARY",
          "DEXTER LAWRENCE",
          "JALEN CARTER",
          "LAIATU LATU",
          "LUKAS VAN NESS"
        ],
        "LB": [
          "JONATHON COOPER",
          "ROQUAN SMITH",
          "ANDREW VAN GINKEL",
          "ARDEN KEY",
          "JALYX HUNT"
        ],
        "DB": [
          "MARSHON LATTIMORE",
          "JI'AYIR BROWN",
          "Cam Taylor-Britt",
          "MIKE SAINRISTIL",
          "REED BLANKENSHIP",
          "CAMRYN BYNUM"
        ]
      }
    },
    {
      "team": "Cowboys",
      "owner": "Tony",
      "roster": {
        "QB": [
          "JORDAN LOVE",
          "ANTHONY RICHARDSON",
          "JAYDEN DANIELS"
        ],
        "RB": [
          "Kenneth Walker III",
          "TYJAE SPEARS",
          "JAYLEN WRIGHT",
          "TANK BIGSBY",
          "MARSHAWN LLOYD"
        ],
        "WR": [
          "DAVANTE ADAMS",
          "DK METCALF",
          "CEEDEE LAMB",
          "ROME ODUNZE",
          "JOSH PALMER",
          "Marvin Mims Jr."
        ],
        "TE": [
          "PAT FREIERMUTH",
          "JAKE FERGUSON",
          "CADE OTTON"
        ],
        "K": [
          "JASON MYERS",
          "GRAHAM GANO"
        ],
        "DL": [
          "KWITY PAYE",
          "MALCOLM KOONCE",
          "DORANCE ARMSTRONG",
          "MYLES MURPHY",
          "MARSHAWN KNEELAND",
          "ALEX WRIGHT"
        ],
        "LB": [
          "BRIAN BURNS",
          "MATTHEW JUDON",
          "JERMAINE JOHNSON II",
          "ARNOLD EBIKETIE",
          "DALLAS TURNER"
        ],
        "DB": [
          "KEVIN BYARD",
          "TRENT MCDUFFIE",
          "JULIAN BLACKMON",
          "QUAN MARTIN",
          "JORDAN WHITEHEAD",
          "TYLER NUBIN"
        ]
      }
    },
    {
      "team": "Colts",
      "owner": "Rizzo",
      "roster": {
        "QB": [
          "JOE BURROW",
          "BROCK PURDY",
          "DRAKE MAYE"
        ],
        "RB": [
          "JAHMYR GIBBS",
          "AARON JONES",
          "DAVID MONTGOMERY",
          "D'ANDRE SWIFT",
          "BLAKE CORUM"
        ],
        "WR": [
          "JA'MARR CHASE",
          "DEVONTA SMITH",
          "AMON-RA ST. BROWN",
          "DJ MOORE",
          "DONTAYVION WICKS",
          "XAVIER LEGETTE"
        ],
        "TE": [
          "DALTON KINCAID",
          "Chig OKONKWO",
          "BRENTON STRANGE"
        ],
        "K": [
          "BRANDON AUBREY",
          "JAKE MOODY"
        ],
        "DL": [
          "Greg ROUSSEAU",
          "QUINNEN WILLIAMS",
          "JEFFERY SIMMONS",
          "CHRISTIAN BARMORE",
          "DARON PAYNE",
          "LEONARD FLOYD"
        ],
        "LB": [
          "KAYVON THIBODEAUX",
          "NOLAN SMITH",
          "JACK CAMPBELL",
          "EDGERRIN COOPER",
          "JONAH ELLISS"
        ],
        "DB": [
          "JAIRE ALEXANDER",
          "DERWIN JAMES",
          "C.J. Gardner-Johnson",
          "L'JARIUS SNEED",
          "AMANI HOOKER",
          "CHRISTIAN GONZALEZ"
        ]
      }
    },
    {
      "team": "Steelers",
      "owner": "Super Dave",
      "roster": {
        "QB": [
          "PATRICK MAHOMES",
          "BRYCE YOUNG",
          "AARON RODGERS"
        ],
        "RB": [
          "BRIAN ROBINSON",
          "BUCKY IRVING",
          "JAYLEN WARREN",
          "RACHAAD WHITE",
          "TREY BENSON"
        ],
        "WR": [
          "TYREEK HILL",
          "LADD MCCONKEY",
          "JAYLEN WADDLE",
          "JORDAN ADDISON",
          "KEENAN ALLEN",
          "JAKOBI MEYERS"
        ],
        "TE": [
          "TRAVIS KELCE",
          "ZACH ERTZ",
          "JONNU SMITH"
        ],
        "K": [
          "WIL LUTZ",
          "JAKE ELLIOTT"
        ],
        "DL": [
          "NICK BOSA",
          "A.J. EPENESA",
          "CAMERON HEYWARD",
          "DEFOREST BUCKNER",
          "MICHAEL DANNA",
          "ED OLIVER"
        ],
        "LB": [
          "KHALIL MACK",
          "JOEY BOSA",
          "ALEX HIGHSMITH",
          "BOBBY WAGNER",
          "KYLE VAN NOY"
        ],
        "DB": [
          "JESSIE BATES",
          "DEREK STINGLEY JR.",
          "A.J. TERRELL",
          "PAULSON ADEBO",
          "BYRON MURPHY",
          "QUINYON MITCHELL"
        ]
      }
    },
    {
      "team": "Vikings",
      "owner": "Denny",
      "roster": {
        "QB": [
          "LAMAR JACKSON",
          "JALEN HURTS",
          "SAM DARNOLD"
        ],
        "RB": [
          "SAQUON BARKLEY",
          "TRAVIS ETIENNE",
          "CHUBA HUBBARD",
          "RHAMONDRE STEVENSON",
          "JORDAN MASON"
        ],
        "WR": [
          "TERRY MCLAURIN",
          "ZAY FLOWERS",
          "QUENTIN JOHNSTON",
          "CHRIS OLAVE",
          "XAVIER WORTHY",
          "KEON COLEMAN"
        ],
        "TE": [
          "SAM LAPORTA",
          "KYLE PITTS",
          "JA'TAVION SANDERS"
        ],
        "K": [
          "CAMERON DICKER",
          "MATT GAY"
        ],
        "DL": [
          "TREY HENDRICKSON",
          "CHASE YOUNG",
          "TRAVON WALKER",
          "CARL GRANDERSON",
          "ZACH ALLEN",
          "SAM WILLIAMS"
        ],
        "LB": [
          "MICAH PARSONS",
          "DAVID OJABO",
          "DERICK HALL",
          "CHRIS BRASWELL",
          "ZAIRE FRANKLIN"
        ],
        "DB": [
          "JAYCEE HORN",
          "DENZEL WARD",
          "KENNY MOORE II",
          "BRIAN BRANCH",
          "JALEN RAMSEY",
          "TERRION ARNOLD"
        ]
      }
    },
    {
      "team": "Jets",
      "owner": "Mario",
      "roster": {
        "QB": [
          "C.J. STROUD",
          "JUSTIN FIELDS",
          "J.J. MCCARTHY"
        ],
        "RB": [
          "JOSH JACOBS",
          "BREECE HALL",
          "TYRONE TRACY JR.",
          "DEVIN SINGLETARY",
          "EMANUEL WILSON"
        ],
        "WR": [
          "NICO COLLINS",
          "DRAKE LONDON",
          "MALIK NABERS",
          "GEORGE PICKENS",
          "ROMEO DOUBS",
          "DARNELL MOONEY"
        ],
        "TE": [
          "EVAN ENGRAM",
          "DARREN WALLER",
          "MIKE GESICKI"
        ],
        "K": [
          "JASON SANDERS",
          "CHRIS BOSWELL"
        ],
        "DL": [
          "Nnamdi Madubuike",
          "DEMARCUS LAWRENCE",
          "CALIJAH KANCEY",
          "ZACH SIELER",
          "JONATHAN ALLEN",
          "ROY ROBERTSON-HARRIS"
        ],
        "LB": [
          "JOSH SWEAT",
          "ODAFE OWEH",
          "JARED VERSE",
          "AZEEZ AL-SHAAIR",
          "Dante Fowler Jr."
        ],
        "DB": [
          "KYLE HAMILTON",
          "XAVIER MCKINNEY",
          "JULIAN LOVE",
          "JOSH JOBE",
          "KEISEAN NIXON",
          "JEVON HOLLAND"
        ]
      }
    },
    {
      "team": "Bengals",
      "owner": "Mass",
      "roster": {
        "QB": [
          "JUSTIN HERBERT",
          "DANIEL JONES",
          "CALEB WILLIAMS"
        ],
        "RB": [
          "DERRICK HENRY",
          "BIJAN ROBINSON",
          "J.K. DOBBINS",
          "JAVONTE WILLIAMS",
          "TYLER ALLGEIER"
        ],
        "WR": [
          "CHRIS GODWIN",
          "GARRETT WILSON",
          "JAXON SMITH-NJIGBA",
          "Deebo Samuel Sr.",
          "JERRY JEUDY",
          "BRANDON AIYUK"
        ],
        "TE": [
          "TREY MCBRIDE",
          "ISAIAH LIKELY",
          "BEN SINNOTT"
        ],
        "K": [
          "DANIEL CARLSON",
          "CHASE MCLAUGHLIN"
        ],
        "DL": [
          "MONTEZ SWEAT",
          "Josh Hines-Allen",
          "MARCUS DAVENPORT",
          "TYREE WILSON",
          "Isaiah McGuire",
          "DARIUS ROBINSON"
        ],
        "LB": [
          "BRADLEY CHUBB",
          "FOYE OLUOKUN",
          "TERREL BERNARD",
          "B.J. OJULARI",
          "NIK BONITTO"
        ],
        "DB": [
          "ANDRE CISCO",
          "TYRIQUE STEVENSON",
          "TARHEEB STILL",
          "JADEN HICKS",
          "NATE WIGGINS",
          "NICK CROSS"
        ]
      }
    },
    {
      "team": "Packers",
      "owner": "Jumps",
      "roster": {
        "QB": [
          "MATTHEW STAFFORD",
          "KYLER MURRAY",
          "TREVOR LAWRENCE"
        ],
        "RB": [
          "ALVIN KAMARA",
          "JOE MIXON",
          "JEROME FORD",
          "RICO DOWDLE",
          "RAY DAVIS"
        ],
        "WR": [
          "TEE HIGGINS",
          "COURTLAND SUTTON",
          "CHRISTIAN WATSON",
          "JAMESON WILLIAMS",
          "PUKA NACUA",
          "MARVIN HARRISON JR."
        ],
        "TE": [
          "T.J. HOCKENSON",
          "NOAH GRAY",
          "HUNTER HENRY"
        ],
        "K": [
          "YOUNGHOE KOO",
          "HARRISON BUTKER"
        ],
        "DL": [
          "AIDAN HUTCHINSON",
          "JAVON HARGRAVE",
          "LEONARD WILLIAMS",
          "BRYCE HUFF",
          "Will McDonald IV",
          "BRADEN FISKE"
        ],
        "LB": [
          "HAROLD LANDRY III",
          "AZEEZ OJULARI",
          "JONATHAN GREENARD",
          "FRANKIE LUVU",
          "BYRON YOUNG"
        ],
        "DB": [
          "KERBY JOSEPH",
          "JAYLON JOHNSON",
          "JALEN PITRE",
          "JOEY PORTER JR.",
          "CALEN BULLOCK",
          "KAMREN KINCHENS"
        ]
      }
    }
  ]
}

In [ ]:
#@title Download Weekly Stats

# Use the league_data_json variable
league_data = league_data_json
season = league_data['season']

# --- 1. Download the CSV Data ---
# Check if the file exists first, if it doesn't then download from
# https://github.com/nflverse/nflverse-data/releases/download/player_stats/stats_player_week_2024.csv (OLD)
# https://github.com/nflverse/nflverse-data/releases/download/stats_player/stats_player_week_2025.csv (NEW)
nfl_weekly_stats_path = f'stats_player_week_{season}.csv'
nfl_weekly_stats_url = f'https://github.com/nflverse/nflverse-data/releases/download/stats_player/stats_player_week_{season}.csv'

print(f"Loading Data For The {season} NFL Season ...")

if not os.path.exists(nfl_weekly_stats_path):
    print(f"{nfl_weekly_stats_path} not found. \nDownloading from NFLverse\n{nfl_weekly_stats_url}...")
    try:
        response = requests.get(nfl_weekly_stats_url)
        response.raise_for_status()  # Raise an exception for bad status codes
        with open(nfl_weekly_stats_path, 'wb') as f:
            f.write(response.content)
        print(f"Successfully downloaded '{nfl_weekly_stats_path}'")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading file: {e}")
        exit()
else:
    print(f"{nfl_weekly_stats_path} found. Using existing file.")




In [ ]:
#@title Get Player ID's
# Load the player stats from the CSV file into a pandas DataFrame
try:
    player_stats_df = pd.read_csv(nfl_weekly_stats_path)
except Exception as e:
    print(f"Error loading '{nfl_weekly_stats_path}' into pandas DataFrame: {e}")
    exit()


# Use the league_data_json variable
league_data = league_data_json
print(f"Matching RFL Rosters Players to Player ID in '{nfl_weekly_stats_path}'")

# --- 2. Process Rosters and Retrieve Player IDs ---

# For easier and case-insensitive matching, create a new column in the DataFrame
# with the player's display name converted to uppercase.
player_stats_df['player_name_upper'] = player_stats_df['player_display_name'].str.upper()

# It's important to work with unique player data from the stats DataFrame
# to avoid issues with duplicate rows for the same player ID.
unique_player_stats_df = player_stats_df.drop_duplicates(subset=['player_id'])


# Lists to store found and not found players
found_players = []
not_found_players = []


# Iterate through each team defined in the JSON file
for team in league_data['teams']:
    team_name = team['team']
    owner_name = team['owner']
    # print(f"--- Processing Team: {team_name} (Owner: {owner_name}) ---") # Removed print statement

    # Get the roster for the current team
    roster = team['roster']

    # Iterate over each position (e.g., "QB", "RB") and the list of players
    for position, players in roster.items():
        # print(f"  {position}:") # Removed print statement
        updated_players = []
        for player_info in players:
            # Check if player_info is a dictionary (already processed) or a string
            if isinstance(player_info, dict) and "Player Name" in player_info:
                player_name = player_info["Player Name"]
            else:
                player_name = player_info

            # Search the unique player DataFrame for a matching player name (case-insensitive)
            player_record = unique_player_stats_df[unique_player_stats_df['player_name_upper'] == player_name.upper()]

            # Check if a record was found
            if not player_record.empty:
                # If multiple matches, try to match by position as well
                if len(player_record) > 1:
                    player_record_by_position = player_record[player_record['position_group'] == position]
                    if not player_record_by_position.empty:
                        player_id = player_record_by_position.iloc[0]['player_id']
                        found_players.append({'RFL Team Name': team_name, 'Position': position, 'Player Name': player_name, 'Player ID': player_id})
                        updated_players.append({"Player Name": player_name, "Player ID": player_id})
                    else:
                        # If no match by position, use the first record and add to not found list
                        player_id = player_record.iloc[0]['player_id']
                        found_players.append({'RFL Team Name': team_name, 'Position': position, 'Player Name': player_name, 'Player ID': player_id})
                        updated_players.append({"Player Name": player_name, "Player ID": player_id})
                        not_found_players.append({'RFL Team Name': team_name, 'Position': position, 'Player Name': player_name, 'Reason': 'Multiple matches, no position match'})
                else:
                    # If only one match, use that record
                    player_id = player_record.iloc[0]['player_id']
                    found_players.append({'RFL Team Name': team_name, 'Position': position, 'Player Name': player_name, 'Player ID': player_id})
                    updated_players.append({"Player Name": player_name, "Player ID": player_id})
                # print(f"    - {player_name}: {player_id}") # Removed print statement
            else:
                # If no match is found, add to not found list and keep the original player name
                not_found_players.append({'RFL Team Name': team_name, 'Position': position, 'Player Name': player_name, 'Reason': 'No match found'})
                if isinstance(player_info, dict) and "Player Name" in player_info:
                     updated_players.append(player_info) # Keep existing dict if not found again
                else:
                    updated_players.append(player_name) # Keep original string if not found


        # Update the roster with the new structure (name and ID)
        team['roster'][position] = updated_players

# Display the count of found players
print(f"Matched {len(found_players)} players out of {len(found_players) + len(not_found_players)}.")

In [ ]:
#@title Missing Player IDs
# Display Players Where a Name was not matched from the league data to the csv file
print(f"{len(not_found_players)} Players Not Matched")
display(pd.DataFrame(not_found_players))

In [ ]:
#@title Create rfl_team_data.csv

# Extract the 'teams' data from the JSON
teams_data = league_data_json['teams']
season = league_data_json['season'] # Get the season

# Create a list to hold flattened team data
flattened_teams = []

# Iterate through each team and its roster to flatten the structure
for team in teams_data:
    team_name = team['team']
    owner = team['owner']
    roster = team['roster']

    for position, players in roster.items():
        for player in players:
            if isinstance(player, dict):
                flattened_teams.append({
                    'Team': team_name,
                    'Owner': owner,
                    'Position': position,
                    'Player Name': player.get('Player Name'),
                    'Player ID': player.get('Player ID'),
                    'Season': season # Add the season
                })
            else:
                 flattened_teams.append({
                    'Team': team_name,
                    'Owner': owner,
                    'Position': position,
                    'Player Name': player,
                    'Player ID': None, # No ID available if it's just a string
                    'Season': season # Add the season
                })

# Create a pandas DataFrame from the flattened data
teams_df = pd.DataFrame(flattened_teams)

# Save the DataFrame to a CSV file
teams_df.to_csv(f'rfl_team_data_{season}.csv', index=False)

print(f"Successfully saved rfl_team_data_{season}.csv")

In [ ]:
#@title Create rfl_scoring_rules.csv

scoring_rules = league_data_json['scoring']
flattened_rules = []

for position, rules in scoring_rules.items():
    for rule_name, details in rules.items():
        flattened_rules.append({
            'Position': position,
            'Rule Name': rule_name,
            'Every': details['every'],
            'Points': details['points']
        })

scoring_rules_df = pd.DataFrame(flattened_rules)
season = league_data_json['season']
filename = f'rfl_scoring_rules_{season}.csv'
scoring_rules_df.to_csv(filename, index=False)
print(f"Successfully saved {filename}")

In [ ]:
#@title Create rfl_roster_rules.csv

roster_rules = league_data_json['roster_rules']
flattened_rules = []

for position, rules in roster_rules.items():
    # Corrected logic to extract roster rules (roster, use, backup)
    flattened_rules.append({
        'Position': position,
        'Roster': rules.get('roster'),
        'Use': rules.get('use'),
        'Backup': rules.get('backup')
    })

roster_rules_df = pd.DataFrame(flattened_rules)
season = league_data_json['season']
filename = f'rfl_roster_rules_{season}.csv'
roster_rules_df.to_csv(filename, index=False)
print(f"Successfully saved {filename}")

In [ ]:
#@title DuckDB Queries

# Python notebook cell
import duckdb, pathlib

# --- Paths (edit if your files live elsewhere) ---
base                = pathlib.Path(".")  # current working directory
CSV_SCORING         = base / f"rfl_scoring_rules_{season}.csv"
CSV_ROSTER_RULES    = base / f"rfl_roster_rules_{season}.csv"
CSV_TEAMS           = base / f"rfl_team_data_{season}.csv"
CSV_STATS           = base / f"stats_player_week_{season}.csv"

# --- Connect (creates rfl.duckdb if it doesn't exist) ---
con = duckdb.connect("rfl.duckdb")

# --- Ingest CSVs into tables ---
con.execute(f"""
CREATE OR REPLACE TABLE rfl_scoring_rules_{season} AS
SELECT * FROM read_csv_auto('{CSV_SCORING.as_posix()}');

CREATE OR REPLACE TABLE rfl_roster_rules_{season} AS
SELECT * FROM read_csv_auto('{CSV_ROSTER_RULES.as_posix()}');

CREATE OR REPLACE TABLE rfl_team_data_{season} AS
SELECT * FROM read_csv_auto('{CSV_TEAMS.as_posix()}');

CREATE OR REPLACE TABLE stats_player_week_{season} AS
SELECT * FROM read_csv_auto('{CSV_STATS.as_posix()}');

CREATE OR REPLACE MACRO chunk_points(v, every, pts) AS
FLOOR(GREATEST(COALESCE(v, 0), 0) / every) * pts;

""")

# --- Create View ---
con.execute(f"""
CREATE OR REPLACE VIEW player_seasonal_fantasy_points_{season} AS
-- Return Total Fantasy Points Per Player for the {season} Season. (Cumulative.)
-- Include the Owner of each player in the results.

WITH player_seasonal_stats AS (
  -- 1. Aggregate all player stats for the regular season.
  -- Sacks are kept as a single total value with decimals (e.g., 3.5).
  SELECT
    t.player_id,
    t.player_display_name,
    td."Position" AS position_group,
    td."Owner" AS owner,
    SUM(passing_yards) AS passing_yards,
    SUM(passing_tds) AS passings_tds,
    SUM(rushing_yards) AS rushing_yards,
    SUM(rushing_tds) AS rushing_tds,
    SUM(receiving_yards) AS receiving_yards,
    SUM(receiving_tds) AS receiving_tds,
    SUM(fg_made) AS fg_made,
    SUM(def_sacks) AS total_sacks, -- Keep total sacks, including fractions
    SUM(def_interceptions) AS def_interceptions,
    SUM(def_tackles_solo + def_tackles_with_assist + def_tackle_assists) AS tackles,
    SUM(def_tds) AS def_tds
  FROM
    stats_player_week_{season} t
  JOIN
    rfl_team_data_{season} td
  ON
    t.player_id = td."Player ID"
  WHERE
    season_type = 'REG'
  GROUP BY
    t.player_id,
    t.player_display_name,
    td."Position",
    td."Owner"
)
-- 2. Join player stats with the original rules and calculate total points
SELECT
  s.player_id,
  s.player_display_name,
  s.position_group,
  s.owner,
  SUM(
    -- Apply the chunk_points macro by dynamically selecting the correct stat
    chunk_points(
      CASE
        r."Rule Name"
        WHEN 'passing_yards' THEN s.passing_yards
        WHEN 'passings_tds' THEN s.passings_tds
        WHEN 'rushing_yards' THEN s.rushing_yards
        WHEN 'rushing_tds' THEN s.rushing_tds
        WHEN 'receiving_yards' THEN s.receiving_yards
        WHEN 'receiving_tds' THEN s.receiving_tds
        WHEN 'fg_made' THEN s.fg_made
        -- For full sacks, only consider the integer part of total_sacks
        WHEN 'full_sack' THEN FLOOR(s.total_sacks)
        -- For half sacks, only consider the fractional part of total_sacks
        WHEN 'half_sack' THEN s.total_sacks - FLOOR(s.total_sacks)
        WHEN 'def_interceptions' THEN s.def_interceptions
        WHEN 'tackles' THEN s.tackles
        WHEN 'def_tds' THEN s.def_tds
        ELSE 0
      END,
      r.Every,
      r.Points
    )
  ) AS total_fantasy_points
FROM
  player_seasonal_stats s
  -- Join stats with the original scoring rules applicable to the player's position
  JOIN rfl_scoring_rules_{season} r ON s.position_group = r.Position
GROUP BY
  s.player_id,
  s.player_display_name,
  s.position_group,
  s.owner;
""")

# --- Display Results from the View ---
display(con.execute(f"""
SELECT * FROM player_seasonal_fantasy_points_{season} ORDER BY total_fantasy_points DESC;
""").df())

In [ ]:
#@title Player Detail [DEBUG]

# Use DuckDB to calculate total fantasy points per team based on roster rules
debug = con.execute(f"""
SELECT  player_id, player_display_name, owner, position_group, total_passing_yards, total_passing_tds, total_rushing_yards,
        total_rushing_tds, total_receiving_yards, total_receiving_tds, total_fg_made, total_full_sacks, total_half_sacks, total_tackles,
        total_interceptions, total_int_tds
from (
    SELECT  player_id,
            sum(passing_yards) AS total_passing_yards, sum(passing_tds) AS total_passing_tds, sum(rushing_yards) as total_rushing_yards,
            sum(rushing_tds) as total_rushing_tds, sum(receiving_yards) as total_receiving_yards, sum(receiving_tds) as total_receiving_tds,
            sum(fg_made) as total_fg_made, floor(sum(def_sacks)) as total_full_sacks, sum(def_sacks) % 1 as total_half_sacks,
            sum(def_tackles_solo+def_tackle_assists) as total_tackles, sum (def_interceptions) as total_interceptions,
            sum(def_tds) as total_int_tds
    FROM stats_player_week_{season}
    WHERE (season_type = 'REG')
    GROUP BY player_id
) AS player_passing_yards_sum
JOIN (
    SELECT DISTINCT player_id, player_display_name, position_group
    FROM stats_player_week_{season}
) AS player_info
USING (player_id)
JOIN (
    SELECT DISTINCT "Player ID" AS player_id, "Owner" AS owner
    FROM rfl_team_data_{season}
) AS team_data
USING (player_id)
""").df()

display(debug)

In [ ]:
#@title Calculate Total Fantasy Points Per Team

# Use DuckDB to calculate total fantasy points per team based on roster rules
total_team_points_df = con.execute(f"""
-- Calculate total fantasy points per team based on roster rules
WITH ranked_players_per_position AS (
    -- Rank players within each team and position by their total fantasy points
    SELECT
        player_id,
        player_display_name,
        position_group,
        owner,
        total_fantasy_points,
        ROW_NUMBER() OVER (PARTITION BY owner, position_group ORDER BY total_fantasy_points DESC) as rank_in_position
    FROM
        player_seasonal_fantasy_points_{season}
),
team_scored_players AS (
    -- Select players who count towards the team's score based on roster rules
    SELECT
        rp.player_id,
        rp.player_display_name,
        rp.position_group,
        rp.owner,
        rp.total_fantasy_points
    FROM
        ranked_players_per_position rp
    JOIN
        rfl_roster_rules_{season} rr ON rp.position_group = rr.Position
    WHERE
        rp.rank_in_position <= rr.Use
)
-- Calculate the total fantasy points for each team
SELECT
    owner,
    SUM(total_fantasy_points) AS total_team_fantasy_points
FROM
    team_scored_players
GROUP BY
    owner
ORDER BY
    total_team_fantasy_points DESC;
""").df()

display(total_team_points_df)

In [ ]:
#@title Create Team Roster with Total Fantasy Points View

con.execute(f"""
CREATE OR REPLACE VIEW team_roster_with_fantasy_points_{season} AS
SELECT
    td."Team",
    td."Owner",
    td."Position",
    td."Player Name",
    psfp.total_fantasy_points
FROM
    rfl_team_data_{season} td
JOIN
    player_seasonal_fantasy_points_{season} psfp ON td."Player ID" = psfp.player_id
ORDER BY
    td."Team",
    td."Position",
    psfp.total_fantasy_points DESC;
""")

# Display the results of the newly created view
display(con.execute(f"""
SELECT * FROM team_roster_with_fantasy_points_{season};
""").df())

In [ ]:
#@title Standings Tree View

# Create the root node for the ipytree
rfl_tree = Tree(nodes=[], opened=False) # Set the root node to be collapsed by default

# Get the unique team owners and sort them alphabetically by total fantasy points descending
# Also get the team name for each owner
owners_teams_df = total_team_points_df.merge(teams_df[['Owner', 'Team']].drop_duplicates(), left_on='owner', right_on='Owner')
owners_teams_df = owners_teams_df.sort_values(by='total_team_fantasy_points', ascending=False)

# Load roster rules for the current season
roster_rules_df = con.execute(f"""
SELECT
    Position,
    Use
FROM
    rfl_roster_rules_{season};
""").df()

# Create a list to hold the jsTree compatible JSON structure (although not displayed, still generated for potential future use)
jstree_data = []

# Iterate through each owner to create a branch for their team
for index, row in owners_teams_df.iterrows():
    owner = row['Owner']
    team_name = row['Team']
    total_points = row['total_team_fantasy_points']

    # Create a node for the team, including their total fantasy points for ipytree
    team_ipynode = Node(f'{owner} [{team_name}] ({total_points:.0f} pts)', opened=False) # Set team nodes to be collapsed

    # Create a node for the team, including their total fantasy points for jsTree
    team_jstree_node = {
        "text": f'{owner} [{team_name}] ({total_points:.0f} pts)',
        "children": []
    }


    # Get the players for the current team from the team_roster_with_fantasy_points view
    team_roster_df = con.execute(f"""
    SELECT
        "Position",
        "Player Name",
        total_fantasy_points
    FROM
        team_roster_with_fantasy_points_{season}
    WHERE
        "Owner" = '{owner}'
    ORDER BY
        "Position",
        total_fantasy_points DESC;
    """).df()

    # Get unique positions for the current team
    positions = sorted(team_roster_df['Position'].unique())

    # Iterate through each position to create a sub-branch
    for position in positions:
        # Get the number of players to use for this position from the roster rules
        use_count = roster_rules_df[roster_rules_df['Position'] == position]['Use'].iloc[0]

        # Create a node for the position, including the number of players used for ipytree
        position_ipynode = Node(f'{position} (Top {use_count})', opened=False) # Set position nodes to be collapsed

        # Create a node for the position, including the number of players used for jsTree
        position_jstree_node = {
            "text": f'{position} (Top {use_count})',
            "children": []
        }


        # Get players for the current position and team
        players_in_position_df = team_roster_df[team_roster_df['Position'] == position]

        # Iterate through each player to create a leaf node
        for index, player_row in players_in_position_df.iterrows():
            player_name = player_row['Player Name']
            player_points = player_row['total_fantasy_points']
            # Create a node for the player with their fantasy points for ipytree
            player_ipynode = Node(f'{player_name} ({player_points:.0f} pts)') # Leaf nodes don't have 'opened' attribute

            # Create a node for the player with their fantasy points for jsTree
            player_jstree_node = {
                "text": f'{player_name} ({player_points:.0f} pts)'
            }
            position_ipynode.add_node(player_ipynode)

            # Add the player node to the position node for jsTree
            position_jstree_node["children"].append(player_jstree_node)


        # Add the position branch to the team branch for ipytree
        team_ipynode.add_node(position_ipynode)

        # Add the position branch to the team branch for jsTree
        team_jstree_node["children"].append(position_jstree_node)

    # Add the team branch to the main ipytree
    rfl_tree.add_node(team_ipynode)

    # Add the team branch to the main jstree data list
    jstree_data.append(team_jstree_node)

# Display the ipytree
display(rfl_tree)

# Save jstree_data to a JSON file
output_filename = f'rfl_standings_{season}.json'
with open(output_filename, 'w') as f:
    json.dump(jstree_data, f, indent=2)

#print(f"Successfully saved jstree data to {output_filename}")

In [ ]:
#@title RFL Standings Graph
import altair as alt

# Merge total_team_points_df with teams_df to get the team name
total_team_points_with_team_df = total_team_points_df.merge(teams_df[['Owner', 'Team']].drop_duplicates(), left_on='owner', right_on='Owner')

chart = alt.Chart(total_team_points_with_team_df).mark_bar().encode(
    x=alt.X('owner', sort='-y', title='Owner [Team]'),  # Sort by total points descending and update axis title
    y=alt.Y('total_team_fantasy_points', title='Total Fantasy Points'), # Add a title to the y-axis
    tooltip=['Owner', 'Team', 'total_team_fantasy_points'] # Include Team in tooltip
).properties(
    title=f'RFL Standings ({season} Season)'
)

# Add text labels for the total points
text = chart.mark_text(
    align='center',
    baseline='bottom',
    dy=-5  # Nudge text above the bar
).encode(
    text=alt.Text('total_team_fantasy_points', format='.0f') # Format as integer
)

# Combine the bar chart and the text
final_chart = chart + text

final_chart.display()

# Save the chart as a PNG file
final_chart.save(f'rfl_standings_{season}.png')

# Print the directory where the file was saved
print(f"Chart saved to: {os.getcwd()}")

In [ ]:
#@title Delete Directory Files (Disabled)
# import pandas as pd
# import json
# import duckdb
# import requests
# import os
# import pathlib

# from google.colab import data_table
# from google.colab import output
# from ipytree import Tree, Node

# data_table.enable_dataframe_formatter()
# output.enable_custom_widget_manager()

# if True:
#   # Get the current directory
#   current_dir = os.getcwd()

#   # List all files and directories in the current directory
#   for item in os.listdir(current_dir):
#       item_path = os.path.join(current_dir, item)
#       try:
#           if os.path.isfile(item_path):
#               os.remove(item_path)
#               print(f"Deleted file: {item_path}")
#           elif os.path.isdir(item_path):
#               shutil.rmtree(item_path)
#               print(f"Deleted directory: {item_path}")
#       except Exception as e:
#           print(f"Error deleting {item_path}: {e}")

#   print("All files and folders deleted.")